In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
import xlrd
import os
import sys
# 경고메세지 숨기기
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
#
from keras.models import load_model

NameError: name 'sys' is not defined

In [ ]:
#kobert라이브러리에서 많이 쓰이는 함수 불러오기
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
#transformers에서 하이퍼파라미터 세팅
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용(권장)
#device = torch.device("cuda:0")
#CPU 사용
device = torch.device("cpu")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

In [7]:
#데이터 불러오기
chatbot_data_pre = pd.read_excel('감성대화말뭉치_table.xlsx')


In [8]:
len(chatbot_data_pre)

5000

In [9]:
#데이터전처리
#데이터구조화 -> 정신건강기반 의미구축 모델을 기반으로 세부감정 -> 분노,불안,우울,자살,긍정 카테고리화

#0  기쁨
#1. 분노 - 안달하는 성가신 툴툴대는 방어적인 혐오스러운
#2. 우울 - 비통한 슬픔 우울한 실망한 눈물이나는 후회되는  버려진
#3. 불안 - 걱정스러운 조심스러운 초조한 두려운 스트레스받는
#4. 자살 - 고립된 상처 염세적인

chatbot_data = chatbot_data_pre.loc[(chatbot_data_pre['감정_대분류'] == "기쁨") | (chatbot_data_pre['감정_소분류'] == "안달하는") | (chatbot_data_pre['감정_소분류'] == "성가신") | (chatbot_data_pre['감정_소분류'] == "툴툴대는") | (chatbot_data_pre['감정_소분류'] == "방어적인") | (chatbot_data_pre['감정_소분류'] == "혐오스러운")
 | (chatbot_data_pre['감정_소분류'] == "비통한") | (chatbot_data_pre['감정_소분류'] == "슬픔") | (chatbot_data_pre['감정_소분류'] == "우울한") | (chatbot_data_pre['감정_소분류'] == "실망한") | (chatbot_data_pre['감정_소분류'] == "후회되는") | (chatbot_data_pre['감정_소분류'] == "눈물이 나는")
 | (chatbot_data_pre['감정_소분류'] == "걱정스러운") | (chatbot_data_pre['감정_소분류'] == "조심스러운") | (chatbot_data_pre['감정_소분류'] == "초조한") | (chatbot_data_pre['감정_소분류'] == "두려운") | (chatbot_data_pre['감정_소분류'] == "스트레스 받는")
 | (chatbot_data_pre['감정_소분류'] == "고립된") | (chatbot_data_pre['감정_소분류'] == "염세적인") | (chatbot_data_pre['감정_소분류'] == "상처") 
 ]

len(chatbot_data)

2428

In [10]:
#데이터 구조 살펴보기
chatbot_data.sample(n=10)

,NO.,성우,연령,성별,상황키워드,감정_대분류,감정_소분류,사람문장1
1489,M_001490,JYW,노년,남성,"건강, 죽음",기쁨,만족스러운,의사가 찬물 말고 따끈한 물을 마시랬는데 효과가 있어. 설사가 많이 줄었어.
978,M_000979,JYS,중년,남성,"대인관계(부부, 자녀)",슬픔,눈물이 나는,일찍 세상에 나온 큰 아들이 인큐베이터에 삼 일을 머물다가 하늘나라로 먼저 갔어.
4177,M_004178,PJS,노년,남성,"건강, 죽음",분노,툴툴대는,괜찮다가 갑자기 가슴통증이 심하고 압박감이 있는 거 보니 곧 심근경색이 올 것 같아.
3929,M_003930,LJB,노년,남성,"건강, 죽음",불안,초조한,요즈음 자꾸 불안한 생각이 들어.
293,M_000294,JYC,청소년,남성,학업 및 진로,상처,고립된,부모님은 내가 시험을 망치면 말도 걸어 주지 않으셔서 너무 슬퍼.
4868,M_004869,YYH,청소년,남성,가족관계,기쁨,흥분,오늘 못 보던 기타가 내 방에 놓여 있어.
166,M_000167,JYC,노년,남성,재정,기쁨,감사하는,대출 연장이 되서 다행이야.
3896,M_003897,LJB,청년,남성,대인관계,불안,초조한,친구와 같은 대학교를 지원했는데 결과 발표가 자꾸 지연돼서 답답하고 불안해.
3283,M_003284,KSS,청소년,남성,학업 및 진로,슬픔,우울한,내 주변의 친구들을 목표를 이루는 데 나는 계속 실패해서 기분이 안 좋아.
3922,M_003923,LJB,노년,남성,"건강, 죽음",불안,초조한,내가 죽으면 내 장례식장에 와서 가족들은 슬퍼하겠지?


In [11]:
# 감정데이터 정수형(수치)로 바꿔주기 (정수 인코딩)
chatbot_data.loc[(chatbot_data_pre['감정_소분류'] == "안달하는") | (chatbot_data_pre['감정_소분류'] == "성가신") | (chatbot_data_pre['감정_소분류'] == "툴툴대는") | (chatbot_data_pre['감정_소분류'] == "방어적인") | (chatbot_data_pre['감정_소분류'] == "혐오스러운"), 'Emotion'] = 1  #분노 => 1
chatbot_data.loc[(chatbot_data_pre['감정_소분류'] == "비통한") | (chatbot_data_pre['감정_소분류'] == "슬픔") | (chatbot_data_pre['감정_소분류'] == "우울한") | (chatbot_data_pre['감정_소분류'] == "실망한") | (chatbot_data_pre['감정_소분류'] == "후회되는") | (chatbot_data_pre['감정_소분류'] == "눈물이 나는"), 'Emotion'] = 2  #우울 => 2
chatbot_data.loc[(chatbot_data_pre['감정_소분류'] == "걱정스러운") | (chatbot_data_pre['감정_소분류'] == "조심스러운") | (chatbot_data_pre['감정_소분류'] == "초조한") | (chatbot_data_pre['감정_소분류'] == "두려운") | (chatbot_data_pre['감정_소분류'] == "스트레스 받는"), 'Emotion'] = 3  #불안 => 3
chatbot_data.loc[(chatbot_data_pre['감정_소분류'] == "고립된") | (chatbot_data_pre['감정_소분류'] == "염세적인") | (chatbot_data_pre['감정_소분류'] == "상처")  , 'Emotion'] = 4  #자살 => 4
chatbot_data.loc[(chatbot_data['감정_대분류'] == "기쁨"), 'Emotion'] = 0  #행복 => 5

/opt/anaconda3/envs/erc/lib/python3.7/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/anaconda3/envs/erc/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [12]:
# 이중리스트형식으로 data_list에 감정과 텍스트 담기
data_list = []
for q, label in zip(chatbot_data['사람문장1'], chatbot_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(int(label)))

    data_list.append(data)

In [16]:
# data_list에 감정,텍스트 형식 살펴보기
print(data_list[0])
print(data_list[600])
print(data_list[1200])
print(data_list[1800])
print(data_list[2000])

['오늘 반장 선거에서 내가 반장이 됐어! 친구들이 날 믿어줘서 너무 고맙다.', '0']
['내가 회사를 그만두면 가족들도 날 떠날 것 같아.', '3']
['지인들과의 모임은 재미있고 앞으로도 계속될 거라 생각해.', '0']
['프레젠테이션 발표에서 한 번 대성공을 거두자 자신감이 붙어 두 번째는 쉬웠어.', '0']
['나 몸이 더 안 좋아지니 갈수록 힘들고 걱정이야.', '3']


In [17]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

# 훈련,테스트데이터 개수 확인
print(len(dataset_train))
print(len(dataset_test))

1821
607


In [18]:
# BERT 모델에 들어가기 위한 dataset을 만들어주는 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [19]:
# 파라미터 세팅 ( 문장길이, 모델배치수,에폭수 ...)
max_len = 50
batch_size = 64
warmup_ratio = 0.1
num_epochs = 12
max_grad_norm = 1
log_interval = 200
learning_rate =  3e-5

In [20]:
#문장을 토크나이저를 통해서 토큰으로 변환(토큰화)
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /Users/moon/Desktop/AI_ERC/ orimodel_test/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [21]:
# 데이터셋 BERT에 맞게 적용
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [22]:
# 텍스트의 토큰화 -> 정수인코딩 결과
data_train[0]

(array([   2, 3096, 5330, 3486,  517, 7827, 7088, 1562, 5776, 5782, 5439,
        5018, 4402, 6060, 4246, 3008, 5760, 3320, 1967, 7096, 3278, 6553,
        3039, 7864, 6855,  517,   54,    3,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1], dtype=int32),
 array(28, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0], dtype=int32),
 2)

In [23]:
#bert모델에 넣기위해 torch형식으로 변환
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [24]:
# KoBERT 학습모델 만들기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [25]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.6).to(device)

In [26]:
#optimizer파라미터 세팅
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [27]:
# 옵티마이저설정(AdamW) 추후에 다른 옵티마이저 확인 필요
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
     

In [28]:
#스케줄 설정
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
     


In [29]:
# 훈련모델의 정확도 계산함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [30]:
# 훈련데이터 형식 확인 
train_dataloader

In [31]:
#데이터 학습모델을 통해 학습 and 테스트
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/opt/anaconda3/envs/erc/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/29 [00:00<?, ?it/s]

[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


epoch 1 batch id 1 loss 1.6394487619400024 train acc 0.328125
epoch 1 train acc 0.31103225326991674


/opt/anaconda3/envs/erc/lib/python3.7/site-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


epoch 12 batch id 1 loss 0.3223847448825836 train acc 0.890625
epoch 12 train acc 0.9055997324613555


  0%|          | 0/10 [00:00<?, ?it/s]

[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


epoch 12 test acc 0.7116935483870968


In [35]:
#classification_report 만들기위해 데이터 전처리
x_pred = []
x_label = []
y_test = []
for i in dataset_test:
  x_pred.append(int(i[1]))
  x_label.append(str(i[0]))

test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)
model.eval()

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)

    valid_length= valid_length
    label = label.long().to(device)

    out = model(token_ids, valid_length, segment_ids)

    test_eval=[]
    for i in out:
        logits=i
        logits = logits.detach().cpu().numpy()
        
        y_test.append(np.argmax(logits))

 # 데이터셋 각 라벨에 대한 정확도 측정
#y_predicted = bilstm_model.predict([X_test])
#pred_tags = f1score.sequences_to_tags(y_predicted)
#test_tags = f1score.sequences_to_tags(y_test)

#print(classification_report(test_tags, pred_tags))

# confusion_matrix 와 classification_report를 통해 결과값 정리
from sklearn.metrics import confusion_matrix, classification_report#

df = pd.DataFrame(columns=['Labels Data','Predicted Labels', 'Actual Labels'])
df['Labels Data'] = x_label
df['Predicted Labels'] = y_test
df['Actual Labels'] = x_pred

df[(df['Actual Labels']==4) & (df['Actual Labels']!=df['Predicted Labels'])].groupby('Predicted Labels').sum()

print(classification_report(x_pred, y_test,target_names=['기쁨','분노','우울','불안','자살']))

[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


              precision    recall  f1-score   support

          기쁨       0.92      0.93      0.92       204
          분노       0.69      0.63      0.66       114
          우울       0.58      0.69      0.63       131
          불안       0.64      0.66      0.65        85
          자살       0.44      0.33      0.38        73

    accuracy                           0.71       607
   macro avg       0.65      0.65      0.65       607
weighted avg       0.71      0.71      0.71       607



In [ ]:
# 경고메세지 숨기기
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [1]:
# 개별 문장을 넣어볼수 있는 (예측하는) 함수
y_test = []
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            #print(logits)
            if np.argmax(logits) == 0:
                test_eval.append("기쁨이")
            elif np.argmax(logits) == 1:
                test_eval.append("분노가")
            elif np.argmax(logits) == 2:
                test_eval.append("우울이")
            elif np.argmax(logits) == 3:
                test_eval.append("불안이")
            elif np.argmax(logits) == 4:
                test_eval.append("자살우려가")
            
        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

# 문장입력기
while True:
  sentence = input("하고싶은 말을 입력해주세요 : ")
  predict(sentence)
  print("\n")

하고싶은 말을 입력해주세요 : dd


NameError: name 'BERTDataset' is not defined

In [ ]:
model.save('kobert_model1110.h5')